In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.linear_model import LinearRegression, LogisticRegression
import duckdb
import requests
from bs4 import BeautifulSoup
import time
import unicodedata

In [38]:
spotify = pd.read_csv('data.csv')
billboard = pd.read_csv('bill.csv')

In [39]:
# Split 'Artist' by 'feat.' or 'ft.' and combine the feature artist into the 'Artist' column
billboard[['Artist', 'FeatureArtist']] = billboard['Artist'].str.split(r'\s+(?:feat\.|ft\.)\s+', n=1, expand=True)
billboard['Artist'] = billboard.apply(lambda row: f"{row['Artist']}, {row['FeatureArtist']}" if pd.notna(row['FeatureArtist']) else row['Artist'], axis=1) 

billboard.drop(columns=['FeatureArtist'], inplace=True)

billboard['Artist'] = billboard['Artist'].str.replace("and ", " ").\
                                            str.strip() 


# Clean Spotify 'name' and 'artists' columns
spotify['name'] = spotify['name'].str.replace(r'\(feat.\..*?\)', '', regex=True).str.replace(r'\(ft.\..*?\)', '', regex=True).str.strip()
spotify['artists'] = spotify['artists'].str.replace("$", "s").\
                                        str.replace("Ø", "o").\
                                        str.replace("and "," ").\
                                        str.strip()



spotify['artists'] = spotify['artists'].str.strip("[]").str.replace("'", "").str.lower()

In [40]:
def remove_accents(text):
    if isinstance(text, str):
        return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    return text

billboard['Artist'] = billboard['Artist'].apply(remove_accents) \
                                    .str.lower() \
                                    .str.replace(r'\(.*?\)', '', regex=True) \
                                    .str.replace(r'[^a-z\s]', '', regex=True) \
                                    .str.strip()

billboard['Song'] = billboard['Song'].apply(remove_accents) \
                                  .str.lower() \
                                  .str.replace(r'\(.*?\)', '', regex=True) \
                                  .str.replace(r'[^a-z\s]', '', regex=True) \
                                  .str.strip()

# Clean and normalize 'Artist Name(s)' and 'Track Name' columns in the Spotify dataset
spotify['artists'] = spotify['artists'].apply(remove_accents) \
                                                      .str.lower() \
                                                      .str.replace(r'\(.*?\)', '', regex=True) \
                                                      .str.replace(r'[^a-z\s]', '', regex=True) \
                                                      .str.strip()

spotify['name'] = spotify['name'].apply(remove_accents) \
                                              .str.lower() \
                                              .str.replace(r'\(.*?\)', '', regex=True) \
                                              .str.replace(r'[^a-z\s]', '', regex=True) \
                                              .str.strip()

billboard['Artist'] = billboard['Artist'].str.lower().str.strip()

In [41]:
billboard['Song'] = billboard['Song'].str.replace('-', ' ') \
                                      .str.replace('/', ' ') \
                                      .str.replace('&', ' ')
                                    

spotify['name'] = spotify['name'].str.replace('-', ' ') \
                              .str.replace('/', ' ') \
                              .str.replace('&', ' ')

spotify['artists'] = spotify['artists'].str.replace("the weeknd", "weeknd").\
                                        str.replace("the everly brothers", "everly brothers").\
                                        str.replace("percy faith  his orchestra", "percy faith")

spotify['name'] = spotify['name'].str.lower().str.strip()
spotify['artists'] = spotify['artists'].str.lower().str.strip()


In [42]:
top_music = duckdb.sql("""
                        SELECT billboard.Year, billboard.Rank, billboard.Artist, billboard.Song, 
                               MEDIAN(spotify.valence) AS valence, MEDIAN(spotify.danceability) AS danceability, 
                               MEDIAN(spotify.energy) AS energy, MEDIAN(spotify.tempo) AS tempo
                        FROM billboard
                        LEFT JOIN spotify ON billboard.Song = spotify.name AND billboard.Artist = spotify.artists
                        GROUP BY billboard.Year, billboard.Artist, billboard.Song, billboard.Rank
                        ORDER BY billboard.Year, billboard.Rank
                    """).df()

# Drop duplicates and save the result
top_music = top_music.drop_duplicates()

In [43]:
no_match = top_music[top_music['valence'].isna()]
print(no_match[['Year', 'Rank', 'Artist', 'Song']].head())
print(no_match.shape) 


after =duckdb.sql("""  SELECT COUNT(*) 
                       FROM no_match
                       WHERE Year < 1970 AND Year >=1960   
                    """).df()

print(after) 
print(no_match) 

    Year  Rank          Artist          Song
3   1960     4  johnny preston  running bear
4   1960     5    mark dinning    teen angel
7   1960     8     jimmy jones     handy man
11  1960    12    bobby rydell      wild one
12  1960    13   brothers four   greenfields
(1949, 8)
   count_star()
0           600
      Year  Rank                        Artist  \
3     1960     4                johnny preston   
4     1960     5                  mark dinning   
7     1960     8                   jimmy jones   
11    1960    12                  bobby rydell   
12    1960    13                 brothers four   
...    ...   ...                           ...   
5571  2016    72                        weeknd   
5584  2016    85      major lazer  dj snake mo   
5585  2016    86  shawn mendes  camila cabello   
5586  2016    87                   fat joe rem   
5593  2016    94   hailee steinfeld  grey zedd   

                                 Song  valence  danceability  energy  tempo  
3        

In [44]:
no_match.to_csv('no_match.csv', index = False) 
# spotify.to_csv('spot.csv', index = False) 
top_music.to_csv('top.csv', index=False)

In [45]:
billboard2 = pd.read_csv('no_match.csv') 
spotify2 = pd.read_csv('top10000song.csv')

In [46]:
spotify2 = spotify2.rename(columns={'Track Name': 'name', 'Artist Name(s)': 'artists'}) 

billboard2.drop(['valence','danceability','energy','tempo'], axis = 1, inplace=True)

In [47]:
def remove_accents(text):
    if isinstance(text, str):
        return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    return text

# Clean and normalize 'Artist' and 'Song' columns in the no_match dataset
billboard2['Artist'] = billboard2['Artist'].apply(remove_accents) \
                                    .str.lower() \
                                    .str.replace(r'\(.*?\)', '', regex=True) \
                                    .str.replace(r'[^a-z\s]', '', regex=True) \
                                    .str.replace("the ","") \
                                    .str.strip()

billboard2['Song'] = billboard2['Song'].apply(remove_accents) \
                                  .str.lower() \
                                  .str.replace(r'\(.*?\)', '', regex=True) \
                                  .str.replace(r'[^a-z\s]', '', regex=True) \
                                  .str.strip()

# Clean and normalize 'Artist Name(s)' and 'Track Name' columns in the Spotify dataset
spotify2['artists'] = spotify2['artists'].apply(remove_accents) \
                                                      .str.lower() \
                                                      .str.replace(r'\(.*?\)', '', regex=True) \
                                                      .str.replace(r'[^a-z\s]', '', regex=True) \
                                                      .str.strip()

spotify2['name'] = spotify2['name'].apply(remove_accents) \
                                              .str.lower() \
                                              .str.replace(r'\(.*?\)', '', regex=True) \
                                              .str.replace(r'[^a-z\s]', '', regex=True) \
                                              .str.strip()

spotify2['artists'] = spotify2['artists'].str.replace("$", "s").\
                                        str.replace("Ø", "o").\
                                        str.replace("and "," ").\
                                        str.replace("the ", "").\
                                        str.strip()


spotify2['name'] = spotify2['name'].str.replace('-', ' ') \
                              .str.replace('/', ' ') \
                              .str.replace('&', ' ') \


merged_df = duckdb.sql("""
                        SELECT billboard2.Year, billboard2.Rank, billboard2.Artist, billboard2.Song, 
                               MEDIAN(spotify2.valence) AS valence, MEDIAN(spotify2.danceability) AS danceability, 
                               MEDIAN(spotify2.energy) AS energy, MEDIAN(spotify2.tempo) AS tempo
                        FROM billboard2
                        LEFT JOIN spotify2 ON billboard2.Song = spotify2.name AND billboard2.Artist = spotify2.artists
                        GROUP BY billboard2.Year, billboard2.Artist, billboard2.Song, billboard2.Rank
                        ORDER BY billboard2.Year, billboard2.Rank
                    """).df()


print(merged_df.shape)
# Checking how many matches were found after cleaning

unmatched_rows = merged_df[merged_df['valence'].isna()]



after =duckdb.sql("""
                       SELECT COUNT(*) 
                       FROM unmatched_rows
                       WHERE Year < 1970 AND Year >= 1960  
                    """).df()
print(after) 



print(unmatched_rows.shape)

(1949, 8)
   count_star()
0           378
(1572, 8)


In [48]:
merged_df.to_csv('second_merge.csv', index = False)

In [49]:
top_df = pd.read_csv('top.csv') 
second_merge_df = pd.read_csv('second_merge.csv') 

In [50]:
merged_df = duckdb.sql("""SELECT top_df.*, 
                            second_merge_df.valence AS valence_second, 
                            second_merge_df.danceability AS danceability_second, 
                            second_merge_df.energy AS energy_second, 
                            second_merge_df.tempo AS tempo_second
                        FROM top_df 
                        LEFT JOIN second_merge_df
                        ON 
                            top_df.Year = second_merge_df.year AND 
                            top_df.Rank = second_merge_df.Rank AND 
                            top_df.Artist = second_merge_df.Artist AND 
                            top_df.Song = second_merge_df.Song
                        ORDER BY top_df.Year, top_df.Rank 
                        """).df() 


merged_df['valence'] = merged_df['valence'].fillna(merged_df['valence_second'])
merged_df['danceability'] = merged_df['danceability'].fillna(merged_df['danceability_second'])
merged_df['energy'] = merged_df['energy'].fillna(merged_df['energy_second'])
merged_df['tempo'] = merged_df['tempo'].fillna(merged_df['tempo_second'])

merged_df.drop(columns=['valence_second', 'danceability_second', 'energy_second', 'tempo_second'], inplace=True)

no_match = merged_df[merged_df['valence'].isna()]

# print(merged_df) 

print(no_match) 


merged_df.to_csv('top_music.csv', index=False)

      Year  Rank                          Artist  \
3     1960     4                  johnny preston   
15    1960    16               hollywood argyles   
26    1960    27  brook benton  dinah washington   
39    1960    40                  freddie cannon   
48    1960    49   hank ballard  the midnighters   
...    ...   ...                             ...   
5568  2016    69            yo gotti nicki minaj   
5584  2016    85        major lazer  dj snake mo   
5585  2016    86    shawn mendes  camila cabello   
5586  2016    87                     fat joe rem   
5593  2016    94     hailee steinfeld  grey zedd   

                                 Song  valence  danceability  energy  tempo  
3                        running bear      NaN           NaN     NaN    NaN  
15                           alleyoop      NaN           NaN     NaN    NaN  
26                               baby      NaN           NaN     NaN    NaN  
39     way down yonder in new orleans      NaN           NaN   

In [51]:
no_match = merged_df[merged_df['valence'].isna()]

# print(merged_df) 

print(no_match) 


merged_df.to_csv('top_music.csv', index=False)

      Year  Rank                          Artist  \
3     1960     4                  johnny preston   
15    1960    16               hollywood argyles   
26    1960    27  brook benton  dinah washington   
39    1960    40                  freddie cannon   
48    1960    49   hank ballard  the midnighters   
...    ...   ...                             ...   
5568  2016    69            yo gotti nicki minaj   
5584  2016    85        major lazer  dj snake mo   
5585  2016    86    shawn mendes  camila cabello   
5586  2016    87                     fat joe rem   
5593  2016    94     hailee steinfeld  grey zedd   

                                 Song  valence  danceability  energy  tempo  
3                        running bear      NaN           NaN     NaN    NaN  
15                           alleyoop      NaN           NaN     NaN    NaN  
26                               baby      NaN           NaN     NaN    NaN  
39     way down yonder in new orleans      NaN           NaN   

In [52]:
clean = merged_df
clean.shape

(5600, 8)

In [53]:
clean = clean.dropna(subset='valence')
clean= clean.dropna(subset='danceability')

In [54]:
countyear= duckdb.sql("""SELECT Year, COUNT(year)
                        FROM clean 
                        GROUP By Year 
                        ORDER BY Year""").df()

countyear 

,Year,"count(""year"")"
0,1960,83
1,1961,76
2,1962,77
3,1963,80
4,1964,41
5,1965,50
6,1966,49
7,1967,51
8,1968,54
9,1969,51
